<a href="https://colab.research.google.com/github/RFJC21/classification_template/blob/master/Classification_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Classification Template**
ANN, SVM, RF, DT

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime, os
import tensorflow as tf
%load_ext tensorboard
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_absolute_error,mean_squared_error, explained_variance_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# Import data

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))
df.head()

# Exploratory Analysis

In [0]:
df.info()

In [0]:
df.describe().transpose()

In [0]:
mask=np.zeros_like(df.corr())
triangle_indices=np.triu_indices_from(mask)
mask[triangle_indices]=True
mask

plt.figure(figsize=(16,10))
sns.heatmap(df.corr(), mask=mask, cmap="coolwarm",annot=True, annot_kws={'size':14})
plt.xticks(fontsize=14)
plt.yticks(fontsize=12)
plt.show()

In [0]:
plt.figure(figsize=(14,8))
df.corr()['Target'].sort_values().drop('Target').plot(kind='bar')

In [0]:
sns.countplot(x='Target',data=df)

In [0]:
plt.figure(figsize=(14,8))
sns.distplot(df['Target'],bins=20, kde=False,color='#1c869c')

In [0]:
plt.figure(figsize=(14,8))
sns.boxplot(x='loan_status',y='loan_amnt',data=df)

In [0]:
plt.figure(figsize=(14,8))
sns.countplot(x='grade',hue='loan_status',order=['A','B','C','D','E','F','G'],data=df)

# Clean Data

In [0]:
df['variable'].value_counts()

In [0]:
(df.isnull().sum()/len(df))*100

In [0]:
df.isnull().sum()

In [0]:
#df['Age']=df['Age'].fillna(df['Age'].mean())

In [0]:
#df=df.dropna()

In [0]:
#df['Cabin'] = df['Cabin'].str[:1]
#df['Cabin'].value_counts()

# Hot Encoding or Dummy Variables

In [0]:
df.select_dtypes(['object']).columns

In [0]:
#1 way
df['loan_repaid']=df['loan_status'].map({'Fully Paid':1,'Charged Off':0})

In [0]:
#dummy variables
subgrade_dummies= pd.get_dummies(df['variable'],drop_first=True)
df=pd.concat([df.drop('variable',axis=1),subgrade_dummies],axis=1)
df.columns

In [0]:
#hot encoding variables
#X=df.iloc[:, :-1].values
#y=df.iloc[:,-1].values

#ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
#X=np.array(ct.fit_transform(X))

# Split Data

In [0]:
df1=df
df2=df
df3=df
df4=df

In [0]:
X=df.drop('Target',axis=1).values
y=df['Target'].values

In [0]:
# P-Values and F-Score

In [0]:
from sklearn.feature_selection import f_classif as fr 

In [0]:
result=fr(X,y)

In [0]:
f_score= result[0]
p_values=result[1]

In [0]:
df.columns

In [0]:
columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'B', 'C',
       'D', 'E', 'F', 'G', 'T']

In [0]:
print ("    Features     ", "F-Score    ", "P-Values")
print ("    -----------  ---------    ---------")

for i in range(0, len(columns)):
    f1 = "%4.2f" % f_score[i]
    p1 = "%2.10f" % p_values[i]
    print("    ", columns[i].ljust(12), f1.rjust(8),"    ", p1.rjust(8))

In [0]:
#if it is a multiclass classification
#hot encoding label
#lb=LabelEncoder()
#y=lb.fit_transform(y)

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=0)

# Scale Data

In [0]:
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# ANN Model

In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1,write_graph=True,write_images=True,profile_batch=2, embeddings_freq=1)

In [0]:
#binary classification
model = Sequential()

# input layer
model.add(Dense(78,  activation='relu'),input_shape=[10,]))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(39, activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(19, activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(units=1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

# optimizer for binary or multiclass= 'msprop'
# for multiclass
# loss='categorical_crossentropy
# activation='softmax'

In [0]:
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=150)

In [0]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=500,
          validation_data=(X_test, y_test), 
          callbacks=[early_stop,tensorboard_callback])

In [0]:
#%tensorboard --logdir logs

In [0]:
loss_df=pd.DataFrame(model.history.history)
loss_df.plot(figsize=(14,8))

In [0]:
loss_df

# Evaluate

In [0]:
predictions=model.predict_classes(X_test)

In [0]:
score_ann=accuracy_score(y_test,predictions)*100
score_ann

In [0]:
print(classification_report(y_test,predictions))

In [0]:
print(confusion_matrix(y_test,predictions))

# Predictions

In [0]:
single_person=df.drop('Target',axis=1).iloc[0]
single_person=scaler.transform(single_person.values.reshape(-1,13))
model.predict_classes(single_person)

In [0]:
df.head()

In [0]:
wine_prediction_ann=df.head(10)
wine_prediction_ann

In [0]:
wine_prediction_ann=scaler.transform(wine_prediction_ann.values.reshape(-1,10))
model.predict_classes(wine_prediction_ann)

# SVM

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [0]:
df1

In [0]:
X=df1.drop('Target',axis=1).values
y=df1['Target'].values

In [0]:
#multiclass classification
#lb=LabelEncoder()
#y=lb.fit_transform(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
classifier = SVC(C= 10,gamma= 0.1,kernel = 'rbf', random_state = 0,probability=True)
classifier.fit(X_train, y_train)

In [0]:
y_pred_svm = classifier.predict(X_test)

In [0]:
dfp=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred_svm})
dfp

In [0]:
score=accuracy_score(y_test,y_pred_svm)*100
score

In [0]:
print(classification_report(y_test,y_pred_svm))

In [0]:
print(confusion_matrix(y_test,y_pred_svm))

# Cross Validation and Grid Search

In [0]:
accuracies_DT = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies_DT.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies_DT.std()*100))

In [0]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10,100], 'kernel': ['linear']},
              {'C': [1, 10,100], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [0]:
wine_prediction_svm=df1.head(10)
wine_prediction_svm

In [0]:
wine_prediction_svm=wine_prediction_svm.drop('Target',axis=1)

In [0]:
wine_prediction_svm = classifier.predict(wine_prediction_svm)
wine_prediction_svm

# Adjust Probability (threshold)

In [0]:
Y_prob = classifier.predict_proba(X_test)[:, 1]

In [0]:
Y_new_pred = []
threshold  = 0.8

for i in range(0, len(Y_prob)):
    if Y_prob[i] > threshold:
        Y_new_pred.append(1)
    else:
        Y_new_pred.append(0)

In [0]:
dfth=pd.DataFrame({'Actual':Y_prob, 'Predicted':Y_new_pred})
dfth

In [0]:
score2=accuracy_score(y_test,Y_new_pred)*100
score2

In [0]:
print(classification_report(y_test,Y_new_pred))

In [0]:
print(confusion_matrix(y_test,Y_new_pred))

# ROC Curve

In [0]:
from sklearn.metrics import roc_curve, roc_auc_score

In [0]:
auc = roc_auc_score(Y_test, Y_prob)

In [0]:
import matplotlib.pyplot as plt

fpr, tpr, threshold = roc_curve(Y_test, Y_prob)
plt.plot(fpr, tpr, linewidth=4)
plt.xlabel("False Positive rate")
plt.ylabel("True Positve Rate")
plt.title("ROC Curve for Loan Prediction")
plt.grid()

# Random Forests

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [0]:
# nao sei se e preciso...
#multiclass classification
#lb=LabelEncoder()
#y=lb.fit_transform(y)

In [0]:
train,test= train_test_split(df2,test_size=0.15, random_state=42)

In [0]:
df2.columns

In [0]:
features=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'B', 'C',
       'D', 'E', 'F', 'G', 'T']

In [0]:
X_train=train[features]
y_train=train['Target']

X_test=test[features]
y_test=test['Target']

In [0]:
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [0]:
rf_model=RandomForestClassifier(bootstrap=False,
 max_depth= 30,
 max_features= 'auto',
 min_samples_leaf= 4,
 min_samples_split= 5,
 n_estimators= 2000,
 random_state=0)
rf_model.fit(X_train,y_train)

In [0]:
rf_pred=rf_model.predict(X_test)

In [0]:
score=accuracy_score(y_test,rf_pred)*100
score

In [0]:
print(classification_report(y_test,rf_pred))

In [0]:
print(confusion_matrix(y_test,rf_pred))

# Grid Search

In [0]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [0]:
 # search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_model, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_

# Decision Trees

In [0]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from IPython.display import SVG  #Scalable Vector Graphics
import graphviz
import io
import pydotplus
from scipy import misc
from graphviz import Source
from IPython.display import display
import imageio

In [0]:
df3

In [0]:
# nao sei se e preciso...
#multiclass classification
#lb=LabelEncoder()
#y=lb.fit_transform(y)

In [0]:
train,test= train_test_split(df3,test_size=0.15)

In [0]:
#numero minimo de casos para criar um novo ramo
c= DecisionTreeClassifier(criterion= 'entropy', max_depth= 9,min_samples_split=50)

In [0]:
df3=df3.drop('Target',axis=1)
#df=df.drop('free sulfur dioxide',axis=1)

In [0]:
df3.columns

In [0]:
features=['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol']

In [0]:
X_train=train[features]
y_train=train['Target']

X_test=test[features]
y_test=test['Target']

In [0]:
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [0]:
dt=c.fit(X_train,y_train)

In [0]:
def show_tree(tree,features,path):
    f=io.StringIO()
    export_graphviz(tree,out_file=f,feature_names=features)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
    img=imageio.imread(path)
    plt.rcParams['figure.figsize']=(50,50)
    plt.imshow(img)

In [0]:
show_tree(dt,features,'dec_tree_1.png')

In [0]:
y_pred=c.predict(X_test)

In [0]:
score=accuracy_score(y_test,y_pred)*100
score

In [0]:
print(classification_report(y_test,rf_pred))

In [0]:
print(confusion_matrix(y_test,rf_pred))

In [0]:
dfp=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
dfp

In [0]:
from sklearn.model_selection import cross_val_score
accuracies_DT = cross_val_score(estimator = c, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies_DT.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies_DT.std()*100))

In [0]:
from sklearn.model_selection import GridSearchCV
parameters = [{'min_samples_split':  range(10,500,20),'max_depth': range(1,20,2)},
              {'min_samples_split':  range(10,500,20), 'criterion': ['entropy'], 'max_depth': range(1,20,2)}]
grid_search = GridSearchCV(estimator = c,
                           param_grid = parameters,
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [0]:
df3=df3.drop('Target',axis=1)

In [0]:
wine_prediction=df.head(10)
wine_prediction

In [0]:
c.predict(wine_prediction)

# Predict on new data

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import io
dftest = pd.read_csv(io.BytesIO(uploaded['wine_predict.csv']))
dftest

In [0]:
dftest=dftest.drop('Target',axis=1)

# RF

In [0]:
rf_pred=rf_model.predict(dftest)

In [0]:
submission = pd.DataFrame({
        "Survived": rf_pred})
submission

In [0]:
data_predicted= pd.concat([df,submission], axis=1)
data_predicted

# ANN

In [0]:
wine_prediction_ann=scaler.transform(dftest.values.reshape(-1,10))

In [0]:
predictions_ann=model.predict_classes(wine_prediction_ann)

In [0]:
df_ANN = pd.DataFrame(predictions_ann, columns=['Predictions_ANN'])
df_ANN

In [0]:
data_predicted= pd.concat([dftest,submission,df_ANN], axis=1)
data_predicted

# Export to CSV

In [0]:
data_predicted.to_csv('predictions.csv')
files.download('predictions.csv')